# Sacramento County Homeless Management Information System Analysis

## Create SQL database
### 5 tables: Clients, Assessment, Programs, Enrollment, Exit

In [1]:
from sqlalchemy import create_engine
from config import username, password
import pandas as pd

In [2]:
# Use sqalchemy to create connection to database and create database tables
engine = create_engine('postgresql://{}:{}@localhost:5432/HMIS_db'.format(username,password))

con = engine.connect()

In [3]:
with engine.connect() as c:
    sql = '''
    DROP TABLE IF EXISTS Clients CASCADE;


CREATE TABLE Clients(
"Race" varchar,
"Ethnicity" varchar,
"Gender" varchar,
"Vet_Status" varchar,
"Vet_Discharge_Status" varchar,
"Created_Date" date,
"Updated_Date" date,
"Birth_Date" date,
"Client_Id" bigint PRIMARY KEY
);

DROP TABLE IF EXISTS Assessment CASCADE;

CREATE TABLE Assessment (
"Client_Id" bigint,
"Assessment_Id" bigint PRIMARY KEY,
"Assessment_Type" varchar,
"Assessment_Score" int,
"Assessment_Date" date,
FOREIGN KEY ("Client_Id") REFERENCES Clients("Client_Id")
);


DROP TABLE IF EXISTS Programs CASCADE;

CREATE TABLE Programs (
"Program_Id" int PRIMARY KEY,
"Agency_Id" int,
"Program_Name" varchar, 
"Program_Start" date,
"Program_End" date,
"Continuum" int,
"Project_Type" varchar,
"Target_Pop" varchar,
"Housing_Type" varchar,
"Added_Date" date,
"Updated_Date" date
);

DROP TABLE IF EXISTS Enrollment CASCADE;

CREATE TABLE Enrollment (
"Client_Id" bigint,
"Enrollment_Id" bigint PRIMARY KEY,
"Household_Id" bigint,
"Program_Id" int,
"Added_Date" date,
"Housing_Status" varchar,
"LOS_Prior" varchar,
"Entry Screen Times Homeless in the Past Three Years" varchar,
"Entry Screen Total Months Homeless in Past Three Years" varchar,
"Zip" int,
"Chronic_Homeless" varchar,
"Prior_Residence" varchar,
"Last_Grade_Completed" varchar,
-- FOREIGN KEY ("Program_Id") REFERENCES Programs("Program_Id"), -program table missing programs
FOREIGN KEY ("Client_Id") REFERENCES Clients("Client_Id")
);

DROP TABLE IF EXISTS Exit_Screen CASCADE; 

CREATE TABLE Exit_Screen (
"Client_Id" bigint,
"Enrollment_Id" bigint,
"Exit_Destination" varchar,
"Exit_Reason" varchar, 
"Exit_Date" date,
FOREIGN KEY ("Client_Id") REFERENCES Clients("Client_Id"),
FOREIGN KEY ("Enrollment_Id") REFERENCES Enrollment("Enrollment_Id")
);

DROP TABLE IF EXISTS Destinations CASCADE;

CREATE TABLE Destinations(
"Destination_Code" int,
"Exit_Destination" varchar PRIMARY KEY
);
    '''
    c.execute(sql)

## Pandas dataframe clean up

In [4]:
# Read data into pandas
assessment = pd.read_csv("Sacramento_County_-_Assessment_Table_2019-09-05T0401_pTq3TT.csv")
client = pd.read_csv("Sacramento_County_-_Client_Table_2019-09-05T0101_Kky8n7.csv")
exit = pd.read_csv("Sacramento_County_-_edit_Exit_Table_2019-09-01T0601_FDwNWs.csv")
enrollment = pd.read_csv("Sacramento_County_-_Enrollment_Table_2019-09-05T0131_KptDcM.csv")
project = pd.read_csv("Sacramento_County_-_Project_Table_2019-09-05T0200_DdZb5N.csv")
destination = pd.read_csv("exit_destinations.csv")

C:\Users\graha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (11,12,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Assessment table

In [5]:
# Eliminate spaces in column names
for i in assessment.columns:
    assessment.rename(columns = {
        i:str(i).replace(' ', '_')
    }, inplace=True)
    
# Rename columns for consistency    
assessment.rename(columns={
    'Personal_ID': 'Client_Id',
    "Assessment_ID":'Assessment_Id'
}, inplace=True)

In [6]:
# Drop unneeded column
assessment.drop(columns=['ignore'], inplace=True)
assessment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8944 entries, 0 to 8943
Data columns (total 5 columns):
Client_Id           8944 non-null int64
Assessment_Id       8944 non-null int64
Assessment_Type     8944 non-null object
Assessment_Score    8944 non-null int64
Assessment_Date     8944 non-null object
dtypes: int64(3), object(2)
memory usage: 349.5+ KB


#### Clients table

In [7]:
client.rename(columns={
    'Clients Race': 'Race',
    'Clients Ethnicity':'Ethnicity',
    'Clients Gender': 'Gender',
    'Clients Veteran Status':'Vet_Status',
    'Clients Discharge Status': 'Vet_Discharge_Status',
    'Clients Date Created Date': 'Created_Date',
    'Clients Date Updated': 'Updated_Date',
    'Birth_Date_d':'Birth_Date',
    'Personal_Id_d':'Client_Id'
},inplace=True)

client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65822 entries, 0 to 65821
Data columns (total 9 columns):
Race                    59828 non-null object
Ethnicity               62972 non-null object
Gender                  64882 non-null object
Vet_Status              51723 non-null object
Vet_Discharge_Status    4708 non-null object
Created_Date            65822 non-null object
Updated_Date            65822 non-null object
Birth_Date              65665 non-null object
Client_Id               65822 non-null int64
dtypes: int64(1), object(8)
memory usage: 4.5+ MB


#### Programs/Projects table

In [8]:
project.rename(columns={
    'Program Id': 'Program_Id',
    'Agency Id': 'Agency_Id',
    'Name': 'Program_Name',
    'Availability Start Date':'Program_Start',
    'Availability End Date': 'Program_End',
    'Continuum Project': 'Continuum',
    'Project Type Code': 'Project_Type',
    'Housing Type':'Housing_Type',
    'Added Date':'Added_Date',
    'Last Updated Date':'Updated_Date',
    'Target Population':'Target_Pop'
}, inplace=True)

# Drop columns labelled as unimportant in source documentation
project.drop(columns=['Unnamed: 0','Affiliated Project Ids','Affiliated with a Residential Project', 'Tracking Method',
                     'Victim Service Provider'], inplace=True)
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 11 columns):
Program_Id       298 non-null int64
Agency_Id        298 non-null int64
Program_Name     298 non-null object
Program_Start    298 non-null object
Program_End      134 non-null object
Continuum        298 non-null int64
Project_Type     298 non-null object
Target_Pop       292 non-null object
Housing_Type     156 non-null object
Added_Date       298 non-null object
Updated_Date     298 non-null object
dtypes: int64(3), object(8)
memory usage: 25.7+ KB


#### Exit table

In [9]:
for i in exit.columns:
    if i == 'Project Exit Date':
        exit.rename(columns={
            i:'Exit_Date'
        }, inplace=True)
        continue
    exit.rename(columns={
        i:str(i).replace(' ', '_')
    }, inplace=True)
exit.rename(columns={
    'Personal_ID':'Client_Id'
}, inplace=True)

exit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135217 entries, 0 to 135216
Data columns (total 5 columns):
Client_Id           135217 non-null int64
Enrollment_Id       135217 non-null int64
Exit_Destination    98426 non-null object
Exit_Reason         108585 non-null object
Exit_Date           135217 non-null object
dtypes: int64(2), object(3)
memory usage: 5.2+ MB


#### Enrollment table

In [10]:
enrollment.rename(columns={
    'Personal ID':'Client_Id',
    'Enrollment Id': 'Enrollment_Id',
    'Household ID': 'Household_Id',
    'Enrollments Project Id': 'Program_Id',
    'Entry Screen Added Date':'Added_Date',
    'Entry Screen Housing Status':'Housing_Status',
    'Entry Screen Length of Stay in Prior Living Situation':'LOS_Prior',
    'Entry Screen Zip Code':'Zip',
    'Entry Screen Chronic Homeless at Project Start':'Chronic_Homeless',
    'Entry Screen Residence Prior to Project Entry':'Prior_Residence',
    'Entry Screen Last Grade Completed':'Last_Grade_Completed'
}, inplace=True)

# Drop columns lablled as unimportant in source documentation
enrollment.drop(columns=['ignore',
                        'Entry Screen Client Became Enrolled in PATH (Yes / No)',
                        'Entry Screen Reason not Enrolled','Entry Screen City','Entry Screen State'
                        ], inplace=True)

In [11]:
enrollment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144067 entries, 0 to 144066
Data columns (total 13 columns):
Client_Id                                                 144067 non-null int64
Enrollment_Id                                             144067 non-null int64
Household_Id                                              144067 non-null int64
Program_Id                                                144067 non-null int64
Added_Date                                                144067 non-null object
Housing_Status                                            91654 non-null object
LOS_Prior                                                 106927 non-null object
Entry Screen Times Homeless in the Past Three Years       43392 non-null object
Entry Screen Total Months Homeless in Past Three Years    32792 non-null object
Zip                                                       70744 non-null float64
Chronic_Homeless                                          144067 non-null object
Prior

##### Delete clients in Exit table who do not have an ID in Client table

In [12]:
exit = exit.loc[((exit['Client_Id'] != 383403592) & (exit['Client_Id'] != 455040993)),:]
exit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135217 entries, 0 to 135216
Data columns (total 5 columns):
Client_Id           135217 non-null int64
Enrollment_Id       135217 non-null int64
Exit_Destination    98426 non-null object
Exit_Reason         108585 non-null object
Exit_Date           135217 non-null object
dtypes: int64(2), object(3)
memory usage: 6.2+ MB


## Write about 350,000 rows to database tables

In [13]:
# Load cleaned up data to database tables
client.to_sql(name="clients", if_exists='append', index=False, con=con)
assessment.to_sql(name="assessment", if_exists='append', index=False, con=con)
project.to_sql(name="programs", if_exists='append', index=False, con=con) 
enrollment.to_sql(name="enrollment",if_exists="append", index=False, con=con)
exit.to_sql(name="exit_screen",if_exists="append", index=False, con=con)
destination.to_sql(name="destinations", if_exists='append', index=False, con=con)

In [14]:
# Create primary key for Exit table 
with engine.connect() as c:
    sql = '''
    ALTER TABLE Exit_Screen
ADD COLUMN ES_Id bigserial PRIMARY KEY;
    '''
    c.execute(sql)

### Creating additional tables then views to use with Flask API 
#### Note data before 2015 deemed unreliable by source

In [15]:
# Table for number of active clients per month 
# Number active = those enrolled in a program without
# an exit date before the end of the queried time period   

dates = pd.date_range(start='1/01/2015',periods=12*5,freq='M')

sql_create = '''
drop table if exists num_active_monthly CASCADE;
create table num_active_monthly (
Act_Date varchar primary key,
Num_Act bigint,
Null_Act bigint
)
'''
with engine.connect() as c:
    c.execute(sql_create)
    
sql_update = '''
insert into num_active_monthly values
('{0}',
(Select count(a."Client_Id")
from enrollment a
left join exit_screen b
on a."Enrollment_Id" = b."Enrollment_Id"
where to_char(a."Added_Date",'YYYY-mm') <= '{0}'
and b."Exit_Date" > '{0}-01'),
(Select count(a."Client_Id")
from enrollment a
left join exit_screen b
on a."Enrollment_Id" = b."Enrollment_Id"
where to_char(a."Added_Date",'YYYY-mm') <= '{0}'
and b."Exit_Date" is null))
'''

for date in dates:
    date = date.strftime('%Y-%m')
    with engine.connect() as c:
            c.execute(sql_update.format(date))

In [16]:
# Table for number of active clients per year
# may change the logic here, currently gets clients who 
# were exclusively active (entered before year start, exited after)

dates_y = ['2015','2016','2017','2018','2019']

sql_create = '''
drop table if exists num_active_yearly CASCADE;
create table num_active_yearly (
Act_Date varchar primary key,
Num_Act bigint,
Null_Act bigint
);
'''
with engine.connect() as c:
    c.execute(sql_create)
    
sql_update = '''
insert into num_active_yearly values
('{0}',
(Select count(a."Client_Id")
from enrollment a
left join exit_screen b
on a."Enrollment_Id" = b."Enrollment_Id"
where to_char(a."Added_Date",'YYYY') <= '{0}'
and b."Exit_Date" > '{0}-01-01'),
(Select count(a."Client_Id")
from enrollment a
left join exit_screen b
on a."Enrollment_Id" = b."Enrollment_Id"
where to_char(a."Added_Date",'YYYY') <= '{0}'
and b."Exit_Date" is null));
'''

for date in dates_y:
    with engine.connect() as c:
        c.execute(sql_update.format(date))

In [17]:
# Create views for exit and entry into programs
with engine.connect() as c:
    sql = '''
    DROP VIEW IF EXISTS monthly_in CASCADE;
    
    create view monthly_in as
    select to_char(e."Added_Date", 'YYYY-mm'), count(e."Client_Id") Num_in
    from enrollment e
    where to_char(e."Added_Date", 'YYYY') > '2014'
    group by to_char(e."Added_Date", 'YYYY-mm')
    order by to_char(e."Added_Date", 'YYYY-mm') desc;
    
    DROP VIEW IF EXISTS monthly_out CASCADE;
    
    create view monthly_out as
    Select to_char(e."Exit_Date", 'YYYY-mm'), count(e."Client_Id") Num_out
    from exit_screen e
    where to_char(e."Exit_Date", 'YYYY') > '2014'
    group by to_char(e."Exit_Date", 'YYYY-mm')
    order by to_char(e."Exit_Date", 'YYYY-mm') desc;
    
    DROP VIEW IF EXISTS yearly_in CASCADE;
    
    create view yearly_in as
    select to_char(e."Added_Date", 'YYYY') date, count(e."Client_Id") Num_in
    from enrollment e
    where to_char(e."Added_Date", 'YYYY') > '2014'
    group by to_char(e."Added_Date", 'YYYY')
    order by to_char(e."Added_Date", 'YYYY') desc;
    
    DROP VIEW IF EXISTS yearly_out CASCADE;
    
    create view yearly_out as
    Select to_char(e."Exit_Date", 'YYYY'), count(e."Client_Id") Num_out
    from exit_screen e
    where to_char(e."Exit_Date", 'YYYY') > '2014'
    group by to_char(e."Exit_Date", 'YYYY')
    order by to_char(e."Exit_Date", 'YYYY') desc;
    '''
    c.execute(sql)

In [18]:
# Create table for top 5 programs for 5 years of data set
dates_y = ['2015','2016','2017','2018','2019']

sql_create = '''
drop table if exists top_5_programs CASCADE;
create table top_5_programs(
"Date" varchar(5),
"Program" varchar(100),
"Num_Enroll" bigint);
'''
with engine.connect() as c:
    c.execute(sql_create)
    
sql_update = '''
insert into top_5_programs

select to_char(e."Added_Date",'YYYY') "Date", p."Program_Name", count(e."Enrollment_Id")"Num_Enroll"
from enrollment e
left join programs p
on p."Program_Id" = e."Program_Id"
where to_char(e."Added_Date",'YYYY') = '{0}'
group by to_char(e."Added_Date",'YYYY'), p."Program_Name" 
order by count(e."Enrollment_Id") desc limit 5;
'''

for date in dates_y:
    with engine.connect() as c:
        c.execute(sql_update.format(date))

## Create Views for Demographics (Gender/Age/Race) per year

In [19]:
with engine.connect() as c:
    sql = '''
update clients
set "Race" = 'Unknown'
where "Race" in ('Client Refused', 'Data Not Collected',
'Client doesn''t Know')
or "Race" is null;

DROP VIEW IF EXISTS yearly_race CASCADE;

create view yearly_race as
select to_char(e."Added_Date",'YYYY') Date, c."Race", count(distinct e."Client_Id") Num_People_Enroll
from enrollment e
left join clients c
on e."Client_Id" = c."Client_Id"
where to_char(e."Added_Date",'YYYY') > '2014'
group by to_char(e."Added_Date",'YYYY'), c."Race"
order by to_char(e."Added_Date",'YYYY'),count(e."Enrollment_Id");

update clients
set "Gender" = 'Unknown'
where "Gender" in ('Client doesn''t know', 'Client refused',
'Data not collected')
or "Gender" is null;

DROP VIEW IF EXISTS yearly_gender CASCADE;

create view yearly_gender as
select to_char(e."Added_Date",'YYYY') Date, c."Gender", count(distinct e."Client_Id") Num_People_Enroll
from enrollment e
left join clients c
on e."Client_Id" = c."Client_Id"
where to_char(e."Added_Date",'YYYY') > '2014'
group by to_char(e."Added_Date",'YYYY'), c."Gender"
order by to_char(e."Added_Date",'YYYY'),count(distinct e."Client_Id");


drop view if exists yearly_age;
create view yearly_age as
select to_char(e."Added_Date",'YYYY') "Date",
((e."Added_Date"::date - c."Birth_Date"::date)/365) "Age"
from enrollment e
left join 
clients c 
on e."Client_Id" = c."Client_Id"
where to_char(e."Added_Date",'YYYY') > '2014'
and c."Birth_Date" is not null
order by "Date", "Age";
'''
    c.execute(sql)

## Create Views/ Tables for number to PH per month (will also need year indication in json object returned from these tables in api). Also views for average time to PH per year. View for number enrolled in a program as homeless per year

In [20]:
sql_create = '''
DROP TABLE IF EXISTS num_to_PH CASCADE;
CREATE TABLE num_to_PH (
Month_Exit varchar PRIMARY KEY,
Num_PH bigint,
Num_Exit bigint
);
'''
with engine.connect() as c:
     c.execute(sql_create)

dates = pd.date_range(start='1/01/2015',periods=12*5,freq='M')

sql_update = '''
INSERT INTO num_to_PH VALUES
('{0}',
(SELECT COUNT (DISTINCT e."Client_Id") 
FROM exit_screen e
LEFT JOIN destinations d
ON e."Exit_Destination" = d."Exit_Destination"
WHERE d."Destination_Code" = 1 
AND to_char(e."Exit_Date", 'YYYY-mm') <= '{0}'
AND e."Exit_Date" > '{0}-01'),
(SELECT COUNT (DISTINCT e."Client_Id") 
FROM exit_screen e
LEFT JOIN destinations d
ON e."Exit_Destination" = d."Exit_Destination"
WHERE to_char(e."Exit_Date", 'YYYY-mm') <= '{0}'
AND e."Exit_Date" > '{0}-01'));
'''

for date in dates:
    date = date.strftime('%Y-%m')
    with engine.connect() as c:
        c.execute(sql_update.format(date))

In [21]:
# Create views for number to PH yearly and all exit yearly
with engine.connect() as c:
    sql = '''
DROP VIEW IF EXISTS yearly_to_ph CASCADE;

CREATE VIEW yearly_to_ph as
SELECT to_char(e."Exit_Date", 'YYYY') date, 
COUNT(e."Client_Id") Num_exit
FROM exit_screen e
LEFT JOIN destinations d
ON e."Exit_Destination" = d."Exit_Destination"
WHERE d."Destination_Code" = 1 
AND to_char(e."Exit_Date", 'YYYY') > '2014'
GROUP BY to_char(e."Exit_Date", 'YYYY')
ORDER BY to_char(e."Exit_Date", 'YYYY') desc;

DROP VIEW IF EXISTS yearly_total_exit CASCADE;
CREATE VIEW yearly_total_exit as
select to_char(e."Exit_Date", 'YYYY') date, 
count(e."Client_Id") Num_exit
from exit_screen e
where to_char(e."Exit_Date", 'YYYY') > '2014'
group by to_char(e."Exit_Date", 'YYYY')
order by to_char(e."Exit_Date", 'YYYY') desc;
    '''
    c.execute(sql)

In [22]:
#Create view for number of unique individuals to programs per year where the client was homeless on entry
from sqlalchemy import text
sql_homeless = text('''
DROP VIEW IF EXISTS yearly_enroll_homeless CASCADE;

create view yearly_enroll_homeless as
select distinct to_char("Added_Date", 'YYYY') "Date",
count(distinct "Client_Id") "Num_Homeless"
from enrollment
where ("Housing_Status" like '%Category 1%' or
"Prior_Residence" = 'Emergency Shelter, including hotel/motel paid for with voucher'
or "Prior_Residence" = 'Hospital or other residential non-psychiatric medical facility'
or "Prior_Residence" = 'Place not meant for habitation'
or "Prior_Residence" = 'Psychiatric hospital or other psychiatric facility'
or "Prior_Residence" = 'Transitional housing for homeless persons')
and to_char("Added_Date", 'YYYY') > '2014'
group by "Date"
order by "Date" desc;
''')


with engine.connect() as connection:
    connection.execute(sql_homeless)

In [23]:
#Create view for average days from entry into transitional housing
# or shelter to permanent housing, for those who get permanent housing
with engine.connect() as c:
    sql = '''
DROP VIEW IF EXISTS avg_to_PH CASCADE;

create view avg_to_PH as 
select distinct to_char(a."Added_Date", 'YYYY') "Date",
avg(b."Exit_Date"::date - a."Added_Date"::date) "Avg_Time_to_PH",
count(distinct a."Client_Id")
from enrollment a
left join exit_screen b
on a."Enrollment_Id" = b."Enrollment_Id"
left join destinations d
on b."Exit_Destination" = d."Exit_Destination"
left join programs p
on a."Program_Id" = p."Program_Id"
where to_char(a."Added_Date", 'YYYY') > '2014'
and d."Destination_Code" = 1
and (p."Project_Type" = 'Transitional Housing'
or p."Project_Type" = 'Day Shelter'
or p."Project_Type" = 'Emergency Shelter')
group by "Date"
'''
    c.execute(sql)

In [24]:
# NEW Create views for % to permanent housing
with engine.connect() as c:
    sql = '''
DROP VIEW IF EXISTS percent_ph_yr CASCADE;

CREATE VIEW percent_ph_yr AS
SELECT p."date" "Date", 
(CAST(p."num_exit" AS FLOAT)/
    CAST(a."num_exit" AS FLOAT)*100) "Percent"
FROM yearly_to_ph p
LEFT JOIN yearly_total_exit a 
ON a."date" = p."date"; 

DROP VIEW IF EXISTS percent_ph_mo CASCADE;

CREATE VIEW percent_ph_mo AS
SELECT "month_exit" "Date", 
(CAST("num_ph" AS FLOAT)/
    CAST(NULLIF("num_exit",0) AS FLOAT)*100) "Percent"
FROM num_to_ph; 
'''
    c.execute(sql)

In [25]:
# Update num_active_yearly and num_active_monthly to have one total column 

with engine.connect() as c:
    sql= '''
alter table num_active_yearly
add column total_act bigint;
update num_active_yearly 
set "total_act" = "null_act" + "num_act";

alter table num_active_monthly
add column total_act bigint;
update num_active_monthly
set "total_act" = "null_act" + "num_act";
'''
    c.execute(sql)

In [26]:
#NEW pandas manipulation for quartiles needed for box plot
        
import numpy as np


sql = 'select * from yearly_age'
con = engine.connect()
age = pd.read_sql(sql=sql, con=con)
for index,row in age.iterrows():
    if row[1] >= 100:
        age.drop([index], inplace=True)
        
        

years = ['2015','2016','2017','2018','2019']
age_stats = {}
for year in years:
    age_stats[year] = age.loc[age.Date == year].describe().T.values


    
age_df = pd.DataFrame(columns=['Count','Mean','std','Min','lower','median','upper','max','date'])
counter = 0
for key in age_stats:
    age_df.loc[counter] = np.append(age_stats[key][0],key)
    counter += 1
    
age_df.to_sql(name='yearly_age_table', if_exists='replace',index=False, con=con)

In [27]:
# UPDATED Changed selecting from yearly age language due to new column names 

with engine.connect() as c:
    sql= '''
drop table if exists monthly_flow;
SELECT A."num_in", 
B."num_out",
C."total_act", C."act_date",
D."num_ph", 
E."Percent" 
into monthly_flow
FROM monthly_in A FULL JOIN monthly_out B ON A."to_char" = B."to_char"
FULL JOIN num_active_monthly C on A."to_char" = C."act_date"
FULL JOIN num_to_ph D on D."month_exit" = A."to_char"
FULL JOIN percent_ph_mo E on E."Date" = D."month_exit"
where A."num_in" is not null
order by "act_date" desc;

drop table if exists yearly_flow;
SELECT A."num_in", 
B."num_out",
C."total_act", C."act_date",
D."num_exit",
E."Avg_Time_to_PH", 
F."Percent"
into yearly_flow
FROM yearly_in A FULL JOIN yearly_out B ON A."date" = B."to_char"
FULL JOIN num_active_yearly C on A."date" = C."act_date"
FULL JOIN yearly_to_ph D on A."date" = D."date"
FULL JOIN avg_to_ph E on E."Date" = A."date"
FULL JOIN percent_ph_yr F on F."Date"=A."date"
order by "act_date" desc;


drop table if exists demographics;
WITH race AS
(
SELECT ROW_NUMBER() OVER(ORDER BY "date") AS ROWNUM, * FROM yearly_race
),
gender AS
(
SELECT ROW_NUMBER() OVER (ORDER BY "date") AS ROWNUM, * FROM yearly_gender
),
age as 
(
SELECT ROW_NUMBER() OVER (ORDER BY "date") AS ROWNUM, * FROM yearly_age_table
),
progs as 
(
SELECT ROW_NUMBER() OVER (ORDER BY "Date") AS ROWNUM, * FROM top_5_programs
)
SELECT 
A."Race", A."num_people_enroll" race_enroll, A."date" RDate,
B."Gender", B."num_people_enroll" gender_enroll, B."date" GDate,
D."Count", D."Min", D."lower",D."median",D."upper", D."max", D."date" ADate,
E."Program", E."Num_Enroll" prog_enroll, E."Date" PDate
into demographics 
FROM race A 
FULL JOIN gender B ON A.ROWNUM = B.ROWNUM
FULL JOIN age D on A.ROWNUM = D.ROWNUM
FULL JOIN progs E on A.ROWNUM = E.ROWNUM;
    '''
    c.execute(sql)

In [28]:
##NEW- make predictions in sql for end of 2019 data 
#simple approach here just using means of previous corresponding months of data 


#monthly table data manipulation 
con = engine.connect()
sql = 'Select * from monthly_flow'
monthly = pd.read_sql(sql=sql,con=con)
replace_dates = ['2019-09','2019-10','2019-11','2019-12']
monthly['month'] = monthly['act_date'].apply(lambda x : str(x).split('-')[1])
monthly.set_index(monthly.act_date, inplace=True)
for date in replace_dates:
    monthly.loc[date,'num_in'] = int(monthly.loc[monthly.month==date.split('-')[1],'num_in'].mean())
    monthly.loc[date,'num_out'] = int(monthly.loc[monthly.month==date.split('-')[1],'num_out'].mean())
    monthly.loc[date,'total_act'] = int(monthly.loc[monthly.month==date.split('-')[1],'total_act'].mean())
    monthly.loc[date,'num_ph'] = int(monthly.loc[monthly.month==date.split('-')[1],'num_ph'].mean())
    monthly.loc[date,'Percent'] = (monthly.loc[date,'num_ph'] / monthly.loc[date,'num_out'])*100
    monthly.loc[date,'act_date'] = date
    

monthly.reset_index(drop=True, inplace=True)
monthly['date'] = pd.to_datetime(monthly['act_date'])
monthly.sort_values('date', inplace=True, ascending=False)


#Yearly flow data table manipulation 
sql = 'Select * from yearly_flow'
yearly = pd.read_sql(sql=sql,con=con)
yearly.loc[yearly.act_date=='2019','num_in'] = monthly.loc[monthly.date.dt.year==2019,'num_in'].sum()
yearly.loc[yearly.act_date=='2019','num_out'] = monthly.loc[monthly.date.dt.year==2019,'num_out'].sum()
yearly.loc[yearly.act_date=='2019','total_act'] = 22000

#write manuipulated dataframes back to db 
yearly.to_sql(name='yearly_flow', con=con, if_exists='replace', index=False)
    #drop created columns needed for making predictions for last months of 2019 
monthly.drop(columns=['date', 'month'], inplace=True)
monthly.to_sql(name='monthly_flow', con=con, if_exists='replace', index=False)
